# Advanced Dataframes

In this lesson we will continue working with pandas DataFrames, and explore some more complex DataFrame manipulations.

In [ ]:
import pandas as pd
import numpy as np

np.random.seed(123)

In [ ]:
# Create list of values for names column.

students = ['Sally', 'Jane', 'Suzie', 'Billy', 'Ada', 'John', 'Thomas',
            'Marie', 'Albert', 'Richard', 'Isaac', 'Alan']

# Randomly generate arrays of scores for each student for each subject.
# Note that all the values need to have the same length here.

math_grades = np.random.randint(low=60, high=100, size=len(students))
english_grades = np.random.randint(low=60, high=100, size=len(students))
reading_grades = np.random.randint(low=60, high=100, size=len(students))

In [ ]:
# Construct the DataFrame using the above lists and arrays.

df = pd.DataFrame({'name': students,
                   'math': math_grades,
                   'english': english_grades,
                   'reading': reading_grades,
                   'classroom': np.random.choice(['A', 'B'], len(students))})

In [ ]:
df.head()

## Creating Dataframes

### From Lists and Dictionaries

There are several ways to create dataframes, we've already seen how we can create a dataframe from a dictionary:

In [ ]:
pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})

The keys in the passed dictionary will be the column names, and the values are the data points that make up each column.

We can also create dataframes from a 2d data structure, either a numpy array or a list of lists:

In [ ]:
pd.DataFrame([[1, 2, 3], [4, 5, 6 ]])

In [ ]:
array = np.array([[1, 2, 3], [4, 5, 6]])

pd.DataFrame(array, columns=['this', 'that', 'the other'])

Notice here that we had to specify the names of the columns ourselves.

### From PyDataset.

In some of the exercises, you'll need to load several datasets using the `pydataset` library. (If you get an error when trying to run the import below, use `pip` to install the `pydataset` package.) When the instructions say to load a dataset from PyDataset, you will need to do the following:

The following import is necessary to access PyDataset datasets:
```python
from pydataset import data
```

Running this code snippet will show you the valuable information doc on the dataset:
```python
data(df_string_name, show_doc=True)
```

Running this code snippet will load the dataset for use as a pandas DataFrame:
```python
df = data(df_string_name)
```

There are 757 available datasets using pydataset. Running the following code snippet in a cell will return a DataFrame with all of your options:
```python
data()
```

In [ ]:
# Load the dataset and store it in the variable mpg.

from pydataset import data
mpg = data('mpg')
mpg.head()

You can view the documentation for the dataset, and any pydata dataset, by setting show_doc to True. This outputs valuable context for your dataset.

```python
data('mpg', show_doc=True) 
```

### From SQL

We can use the `read_sql` method to create a dataframe based on the results of a SQL query. To do this, we need to tell pandas how to connect to the database we are querying. The way we communicate this to pandas is with a specially formatted *connection string*.

In addition, whenever we want to connect to a database from our python code (other programming languages are similar), we will need a **driver**, a bit of software that handles the details of the database connection.

In order to connect to mysql, we'll install the `pymysql` driver packages:

`python -m pip install pymysql`

Once those are installed, we can create the connection string. In general, database connection urls will have this format:

```python
protocol://[user[:password]@]hostname/[database_name]
```

Here's an example of what one would look like:

```python
mysql+pymysql://codeup:p@assw0rd@123.123.123.123/some_db
```

Another thing we need to consider is that we don't want to publish our database credentials to github, however, we will need access to these values in our code in order to create the connection string defined above.

In order to accomplish this, we can define several variables in a file named `env.py` that contain the sensitive data, add `env.py` to our `.gitignore` file, and then import those values into another script. 

**Be 100% sure to add `env.py` to this specific repository's `.gitignore` file, even and especially, if you have already added `env.py` to your global .gitignore file. This will protect the env file for people who clone this project (like collaborators)**

In [ ]:
# pd.read_sql(query, url)

In [ ]:
from env import host, user, password

url = f'mysql+pymysql://{user}:{password}@{host}/employees'

Once this url is defined, we can use it with the `read_sql` function to have pandas treat the results of a SQL query as a dataframe.

In [ ]:
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', url)

It is common to have longer SQL queries that we want to read into python, and an example of how we might break a query into several lines is below:

In [ ]:
sql = '''
SELECT
    emp_no,
    first_name,
    last_name
FROM employees
WHERE gender = 'F'
LIMIT 100
'''

employees = pd.read_sql(sql, url)
employees.head()

!!!danger "Passwords and Sensitive Information"
    Don't add and commit files with passwords or other sensitive information in them to a git repository!

In [ ]:
query = '''
SELECT
    t.title as title,
    d.dept_name as dept_name
FROM titles t
JOIN dept_emp USING (emp_no)
JOIN departments d USING (dept_no)
LIMIT 100
'''

title_dept = pd.read_sql(query, url)
title_dept.head()

## Exercises I

Run `python -m pip install pymysql` from your terminal to install pymysql.

Create a notebook or python script named `advanced_dataframes` to do your work in for these exercises.

1. Run `python -m pip install pymysql` from your terminal to install the mysql client (any folder is fine)
1. cd into your exercises folder for this module and run `echo env.py >> .gitignore`
1. Create a function named `get_db_url`. It should accept a username, hostname, password, and database name and return a url connection string formatted like in the example at the start of this lesson.

2. Use your function to obtain a connection to the `employees` database.

3. Once you have successfully run a query:

    a. Intentionally make a typo in the database url. What kind of error message do you see?
    
    b. Intentionally make an error in your SQL query. What does the error message look like?

4. Read the `employees` and `titles` tables into two separate DataFrames.

5. How many rows and columns do you have in each DataFrame? Is that what you expected?

6. Display the summary statistics for each DataFrame.

7. How many unique titles are in the `titles` DataFrame?

8. What is the oldest date in the `to_date` column? 

9. What is the most recent date in the `to_date` column?

## Indexing and Subsetting

Like the pandas Series object, the pandas DataFrame object supports both position- and label-based indexing using the indexing operator `[]`.

I will demonstrate concrete examples of indexing using the indexing operator `[]` alone and with the `.loc` and `.iloc` attributes below.

### `[]`

I can pass a list of columns from a DataFrame to the indexing operator (aka bracket notation) to return a subset of my original DataFrame.

In [ ]:
# Choose only two columns for my subset.

df[['name', 'classroom']].head()

In [ ]:
# I can pass a boolean Series to the indexing operator as a selector.

bools = df.name.str.startswith('A')
bools

In [ ]:
df[bools]

### `.loc`

We can use the `.loc` attribute to select specific rows AND columns by index label. The index label can be a number, but it can also be a string label. This method offers a lot of flexibility! **The .loc attribute's indexing is inclusive and uses an index label, not position.**

```python
df.loc[row_indexer, column_indexer]
```

In [ ]:
df.head()

In [ ]:
# Select all the rows and a subset of columns; notice the inclusive behavior of the indexing.

df.loc[:, 'math':'reading']

### `.iloc`

We can use the `.iloc` attribute to select specific rows and colums by index position. `.iloc` does not accept a boolean Series as a selector like `.loc` does. **It takes in integers representing index position and is NOT inclusive.**

```python
df.iloc[row_indexer, column_indexer]
```

We can select rows by integer position:

In [ ]:
# Notice the exclusive behavior of the indexing.

df.iloc[:3]

We can also specify which columns we want to select:

In [ ]:
df.iloc[:3, 1:3]

Here we select the first 3 rows (everything up to but not including the index of 3), and the second and third columns (starting from the index of 1 up to but not including the index of 3).

## Aggregating

### `.agg`

The `.agg` method lets us specify a way to aggregate a series of numerical values. We pass an aggregate function or list of functions to the method that we want applied to a Series.

In [ ]:
df.math.median()

In [ ]:
df.math.min()

In [ ]:
df.reading.agg('min')

While on the surface this seems pretty simple, `.agg` is capable of providing more detailed aggregations:

In [ ]:
df.head()

In [ ]:
df[['english', 'reading', 'math']].mean()

In [ ]:
df[['english', 'reading', 'math']].agg(['mean', 'min', 'max'])

### `.groupby`

The `.groupby` method is used to create a grouped object, which we can then apply an aggregation on. For example, if we wanted to know the highest math grade from each classroom:

In [ ]:
df[["classroom", "math", "reading", "english"]].groupby("classroom").max()

In [ ]:
df.groupby('classroom').math.max()

We can use `.agg` here to, to see multiple aggregations:

In [ ]:
df.groupby('classroom').math.agg(['min', 'mean', 'max'])

In [ ]:
df.head()

We can group by multiple columns as well. To demonstrate, we'll create a boolean column named `passing_math`, then group by the combination of our new feature, `passing_math`, and the classroom and calculate the average reading grade and the number of individuals in each subgroup. 

Let's break this problem down and code it step-by-step.

#### `np.where`

First, we can create the new `passing_math` column using a handy NumPy function called `np.where`. It will allow us to base the new column values on whether the values in an existing column, `math`, meet a condition.

```python
np.where(condition, this_where_True, this_where_False)
```

In [ ]:
# Create the new column based on an existing column.

df['passing_math'] = np.where(df.math < 70, 'failing', 'passing')
df

Next, we will group by the `passing_math` and `classroom` columns and use the `.agg` method to calculate the average reading grade and the number of students.

In [ ]:
grade_groups = df.groupby(['passing_math', 'classroom']).reading.agg(['mean', 'count'])
grade_groups

In [ ]:
# I can even clean up my columns to make my calculations clearer.

grade_groups.columns = ['avg_reading_grade', 'count_of_students']
grade_groups

**Takeaways:**

We can interpret this output as there being 2 students failing math in classroom A with an average reading grade of 87, 6 students passing math in classroom A with an average reading grade of 87.16, and 4 students passing math in classroom B with an average reading grade of 85.25.

## Merging and Joining

Pandas provides several ways to combine dataframes together. We will look at two of them below:

### `pd.concat`

This function takes in a list or dictionary of Series or DataFrame objects and joins them along a particular axis, row-wise axis=0 or column-wise axis=1.

```python
# For example, concat with a list of two DataFrames
pd.concat([df1, df2], axis=0)
```

- When your list contains at least one DataFrame, a DataFrame is returned.


- When concatenating only Series objects row-wise, axis=0, a Series is returned.


- When concatenating Series or DataFrames column-wise, axis=1, a DataFrame is returned.

```python
# Default is set to row-wise concatenation using an outer join.
pd.concat(objs, axis=0, join='outer')
```

When concatenating dataframes vertically, we basically are just adding more rows to an existing dataframe. In this case, the dataframes we are putting together should have the same column names[^1].

In [ ]:
"con" + "cat" + "e" + "nation"

In [ ]:
df1 = pd.DataFrame({'a': [1, 2, 3]})
df2 = pd.DataFrame({'a': [4, 5, 6]})

df1

In [ ]:
df2

In [ ]:
pd.concat([df1, df2])

**Note** that the indices are preserved on the resulting dataframe; we could set the `ignore_index` parameter to `True` if we wanted these to be sequential.

In [ ]:
concat_df1 = pd.concat([df1, df2], ignore_index=True)
concat_df1

[^1]:
    We can concatenate dataframes with different column names, but generally this is not the behavior we want, as pandas will fill in a lot of null values into the resulting dataframe. The exception to this is if the dataframes are aligned on their index (i.e. the labels for each row), then we can provide the `axis=1` keyword argument to `pd.concat` to merge the dataframes horizontally.

In [ ]:
concat_df2 = pd.DataFrame({'b': [1, 2, 3, 4, 5, 6]})
concat_df2

In [ ]:
# Axis=1 means concat horizontally
pd.concat([concat_df1, concat_df2], axis=1)

### `.merge`

This method is similar to a SQL join. Here's a [cool read](https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_sql.html#compare-with-sql-join) making a comparison between the two, if you're interested.

```python
# df.merge default settings for commonly used parameters.

left_df.merge(right_df, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, indicator=False)
```

How does changing the default argument of the `how` parameter change my resulting DataFrame?

`how` == Type of merge to be performed.

`how=left`: use only keys from left frame, similar to a SQL left outer join; preserve key order.

`how=right`: use only keys from right frame, similar to a SQL right outer join; preserve key order.

`how=outer`: use union of keys from both frames, similar to a SQL full outer join; sort keys lexicographically.

`how=inner`: use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys.

In [ ]:
# Create the users DataFrame.

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

In [ ]:
# Create the roles DataFrame

roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles

The `.merge` method will allow us to specify `left_on` and `right_on` to indicate the columns that are the keys used to merge the dataframes together. 

- In addition, the `how` keyword argument is used to define what type of JOIN we want to do; as we saw above, `inner` is the default setting. 

- For demonstration purposes, I'm also going to set the `indicator` parameter to `True`, which will create a column indicating whether the merge key appears in the `left_only`, `right_only` or `both` DataFrames.

In [ ]:
# Perform an outer join specifying the left and right DataFrame keys.

users.merge(roles, left_on='role_id', right_on='id', how='outer', indicator=True)

Notice that we have duplicate column names in the resulting dataframe. By default, pandas will add a suffix of `_x` to any columns in the left dataframe that are duplicated, and `_y` to any columns in the right dataframe that are duplicated. I can clean up my columns if I want to; one way would be to use method chaining, which it demonstrated below:

In [ ]:
# Inner join tolerates no nulls.
# Inner join only shows user who have roles, roles that have users
users.merge(roles, how="inner", left_on="role_id", right_on="id")

In [ ]:
(users.merge(roles, 
            left_on='role_id', 
            right_on='id', 
            how='outer')
    .drop(columns='role_id')
    .rename(columns={'id_x': 'id', 
                     'name_x': 'employee',
                     'id_y': 'role_id',
                     'name_y': 'role'}
            )
)

## Exercises II

1. Copy the `users` and `roles` DataFrames from the examples above. 

2. What is the result of using a `right` join on the DataFrames? 

3. What is the result of using an `outer` join on the DataFrames?
     
4. What happens if you drop the foreign keys from the DataFrames and try to merge them?

5. Load the `mpg` dataset from PyDataset. 

6. Output and read the documentation for the `mpg` dataset.

7. How many rows and columns are in the dataset?

8. Check out your column names and perform any cleanup you may want on them.

9. Display the summary statistics for the dataset.

10. How many different manufacturers are there?

11. How many different models are there?

12. Create a column named `mileage_difference` like you did in the DataFrames exercises; this column should contain the difference between highway and city mileage for each car.

13. Create a column named `average_mileage` like you did in the DataFrames exercises; this is the mean of the city and highway mileage.

14. Create a new column on the `mpg` dataset named `is_automatic` that holds boolean values denoting whether the car has an automatic transmission.

15. Using the `mpg` dataset, find out which which manufacturer has the best miles per gallon on average?

16. Do automatic or manual cars have better miles per gallon?

## Reshaping

We will talk about reshaping operations in more detail when we discuss tidy data, but for now we will focus on a couple of common operations that can be used to summarize our data by different subgroups.

### `pd.crosstab`

For an example of `.crosstab`, we will count the number of students passing math in each classroom.

In [ ]:
# We will use our student grades DataFrame, df.

df

We'll use the `pd.crosstab` function to count the number of occurances of each subgroup (i.e. each unique combination of classroom and whether or not the student is passing math):

In [ ]:
pd.crosstab(df.passing_math, df.classroom)

We can also view subtotals with the `margins` set to `True`.

In [ ]:
pd.crosstab(df.passing_math, df.classroom, margins=True)

The `.crosstab` function will let us view the numbers as percentages of the total as well by setting `normalize` to `True`.

In [ ]:
pd.crosstab(df.passing_math, df.classroom, normalize=True).round(3)

### `.pivot_table`

Here we use the `.pivot_table` method to create our summary. This method produces output similar to an excel pivot table. We must supply 3 things here:

- which values will make up the rows (the `index`)
- which values will make up the columns
- the values we are aggregating
- an aggregation method (`aggfunc`); if we can omit this, and `mean` will be used by default

For an example using the `pivot_table` method, we'll calculate the average math grade for the combination of `classroom` and `passing_math` status.

In [ ]:
df.pivot_table(index='classroom', columns='passing_math', values='math')

Here we'll create a dataframe that represents various orders at a restaurant.

In [ ]:
n = 40

orders = pd.DataFrame({
    'drink': np.random.choice(['Tea', 'Water', 'Water'], n),
    'meal': np.random.choice(['Curry', 'Yakisoba Noodle', 'Pad Thai'], n),
})

orders.sample(10)

#### `.map`

The `.map` method lets us use a dictionary to calculate the total price for an order; then I can save my calculations to a new column named `bill`. Let's do this step-by-step.

In [ ]:
# Create a dictionary of prices for drinks and meals.

prices = {
    'Yakisoba Noodle': 9,
    'Curry': 11,
    'Pad Thai': 10,
    'Tea': 2,
    'Water': 0,
}

In [ ]:
"""
Match the values in the 'drink' and 'meal' columns with the values in the 'prices' dictionary 
and perform the specified calculation. Save this calculation to a new column named 'bill'.
"""

orders['bill'] = orders.drink.map(prices) + orders.meal.map(prices)

orders.sample(10)

Let's take a look at how many orders have each combination of meal and drink:

In [ ]:
pd.crosstab(orders.drink, orders.meal)

In [ ]:
pd.crosstab(orders.drink, orders.meal, normalize=True, margins=True)

And let's find out the average bill amount for each combination: 

In [ ]:
orders.pivot_table(index='drink', columns='meal', values='bill')

It's interesting to note that we could find the same information with a multi-level group by:

In [ ]:
orders.groupby(['drink', 'meal']).bill.mean()

The choice between group by and a pivot table here is mostly asthetic, and you should use whichever makes more sense to you with the problem at hand. 

### Transposing

In [ ]:
df.T

In [ ]:
df.describe().T

## Exercises III

1. Use your `get_db_url` function to help you explore the data from the `chipotle` database. 

2. What is the total price for each order?

3. What are the most popular 3 items?

4. Which item has produced the most revenue?

5. Join the `employees` and `titles` DataFrames together.

6. For each title, find the hire date of the employee that was hired most recently with that title.

7. Write the code necessary to create a cross tabulation of the number of titles by department. (Hint: this will involve a combination of SQL code to pull the necessary data and python/pandas code to perform the manipulations.)

### Extra Pandas Exercises and Resources
- [https://www.w3resource.com/python-exercises/pandas/index.php](https://www.w3resource.com/python-exercises/pandas/index.php)
- [https://towardsdatascience.com/20-pandas-functions-that-will-boost-your-data-analysis-process-f5dfdb2f9e05](https://towardsdatascience.com/20-pandas-functions-that-will-boost-your-data-analysis-process-f5dfdb2f9e05)
- [https://github.com/guipsamora/pandas_exercises](https://github.com/guipsamora/pandas_exercises)
- [https://github.com/ajcr/100-pandas-puzzles](https://github.com/ajcr/100-pandas-puzzles)

### More Practice!

For even more practice with pandas, you can do the exercises from the SQL module, but instead of using SQL to do the aggregation, sorting, joining, etc, use pandas. That is, read the data from all of the tables into pandas dataframes and manipulate the dataframes.